Ayda is working on presidential organization of Azerbaijan and she is ordered to remove all comments, both in Azerbaijani and English about racism (or other types of sarcasms) automatically from the President's twitter (or Instagram or Facebook) page. As there are lots of such comments she should create a classification model to do so.
Please help her in finding such comments. You are allowed to use whatever tool you want but for the classification, you are just allowed to use LR to do so.
Hint: Use translation python tools like 'mtranslate' for translation.

Good luck.


In [ ]:
import os

user = "nabiyevnabi"
key = "c048717f48dcbd89343bf759a45f8c68"
if '.kaggle' not in os.listdir('/root'):
    !mkdir ~/.kaggle
!touch /root/.kaggle/kaggle.json
!chmod 666 /root/.kaggle/kaggle.json
with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write('{"username":"%s","key":"%s"}' % (user, key))
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d rmsharks4/sarcasmania-dataset

  0% 0.00/1.59M [00:00<?, ?B/s]
100% 1.59M/1.59M [00:00<00:00, 53.1MB/s]


In [ ]:
!unzip sarcasmania-dataset.zip

Archive:  sarcasmania-dataset.zip
  inflating: sarcasmania-dataset.csv  


In [ ]:
import pandas as pd
import numpy as np
all_comments=pd.read_csv('/content/sarcasmania-dataset.csv')

In [ ]:
pip install googletrans

     |████████████████████████████████| 61kB 2.7MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 962kB 8.4MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 102kB 9.7MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-cp36-none-any.whl size=15736 sha256=a720d75ebcec7fd73082fa4d80139e097cce9a41b5abed84f39d03931d108c52
  Stored in directory: /root/.cache/pip/wheels/28/1a/a7/eaf4d7a3417a0c65796c547cff4deb6d79c7d14c2abd29273e
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=4f3e89ad182e9b2e2f89f5ca8f21599a09c03934acf27a957f580014ff55307e
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built googletrans contextvars


Let's test the translate function

In [ ]:
from googletrans import Translator,constants
translator=Translator()
translation=translator.translate("Sabir 2100 manat qazanir",src='az')
print(translation.text)

Sabir earns 2,100 manats


In [ ]:
translation=translator.translate(all_comments.iloc[8,1],src='en',dest='az')
print(translation.text)

Bu səhər 10 km piyada gedin. Zəhmli bir iş gördük.


So,the translator is quite good

In [ ]:
all_comments

,id,tweet,sarcasm,humor,insult
0,1,@0430yes i hope youre lurking rn. i want to li...,0,0,0
1,2,05 really taught me a valuable lesson I'm neve...,0,1,1
2,3,"@098BERRY Never had a voice to protest, so you...",0,0,0
3,4,@0hMySt4rs Rest in peace & love to you and you...,0,0,1
4,5,100 days until Christmas! 🌲 #too soon #not rea...,0,0,1
...,...,...,...,...,...
39775,39776,@Zendaya I could see the makeup artists giving...,1,0,1
39776,39777,@ZiggiWatkins11 Slvr... That's great name #NOT...,1,1,0
39777,39778,@zoso4986 @Nero He is the fag we need but not ...,1,1,1
39778,39779,Zuma sounding like Kanye West right now trying...,1,0,1


Shuffling the dataset

In [ ]:
np.random.seed(128)
shuffled_comments=all_comments.sample(frac=1).reset_index(drop=True)
shuffled_comments

,id,tweet,sarcasm,humor,insult
0,34247,RT I'm Not Saying that #Hitler Was A Great Guy...,1,1,1
1,36241,Texting is cool. You sound polite to the recei...,1,0,1
2,4020,Eraticus take a sickle and bring back sprouts :),0,0,1
3,24509,"@GetIt_Back20 u know me, im nothing like "" the...",1,0,0
4,35193,seriously my favorite thing is when people rem...,1,1,1
...,...,...,...,...,...
39775,8632,"I love your voice, and your Eyes, and your smi...",0,0,1
39776,21618,After a 8am meeting now to domestic abuse trai...,1,0,0
39777,9530,I sleep with two knives in my room,0,1,0
39778,30182,kkk are the police are the kkk are the police....,1,1,1


I will pick 1000 samples to make translation process *faster*

In [ ]:
shuffled_comments=shuffled_comments.iloc[:1000,:]

In [ ]:
az_tweets=[]
for tweet in shuffled_comments['tweet']:
  translation=translator.translate(tweet,src='en',dest='az')
  az_tweets.append(translation.text)

In [ ]:
sarcasm=shuffled_comments.iloc[:1000,2]
humor=shuffled_comments.iloc[:1000,3]
insult=shuffled_comments.iloc[:1000,4]

Creating the dataset,that contains only translated tweets

In [ ]:
azerbaijani_dataset=pd.DataFrame({'tweet':az_tweets,'sarcasm':sarcasm,'humor':humor,'insult':insult})

In [ ]:
azerbaijani_dataset

,tweet,sarcasm,humor,insult
0,RT # Hitlerin Möhtəşəm Bir Adam olduğunu Demir...,1,1,1
1,Mesaj yazmaq yaxşıdır. Alıcına nəzakətli səs v...,1,0,1
2,Eraticus bir oraq götürüb cücərtilər gətirir :),0,0,1
3,"@ GetIt_Back20 məni tanıyırsınız, ""onlar"" kimi...",1,0,0
4,ciddi mənim ən sevdiyim şey insanlar dedikləri...,1,1,1
...,...,...,...,...
995,LAX vasitəsilə bu qaçış üçün tamamilə enerjim ...,1,0,0
996,Ah yes that was a fantastic feeling let's dig ...,0,0,0
997,mia bir selfini sevir .. # deyil # sərcasm,1,1,0
998,"Şampan, səhər yeməyi, gəzinti və sonra bira pi...",0,1,1


I will work with 10000 tweets,there will be 9000 original tweets and 1000 translated tweets

In [ ]:
all_comments=all_comments.iloc[:9000,:]

In [ ]:
overall=pd.concat([all_comments,azerbaijani_dataset])

In [ ]:
overall

,id,tweet,sarcasm,humor,insult
0,1.0,@0430yes i hope youre lurking rn. i want to li...,0,0,0
1,2.0,05 really taught me a valuable lesson I'm neve...,0,1,1
2,3.0,"@098BERRY Never had a voice to protest, so you...",0,0,0
3,4.0,@0hMySt4rs Rest in peace & love to you and you...,0,0,1
4,5.0,100 days until Christmas! 🌲 #too soon #not rea...,0,0,1
...,...,...,...,...,...
995,NaN,LAX vasitəsilə bu qaçış üçün tamamilə enerjim ...,1,0,0
996,NaN,Ah yes that was a fantastic feeling let's dig ...,0,0,0
997,NaN,mia bir selfini sevir .. # deyil # sərcasm,1,1,0
998,NaN,"Şampan, səhər yeməyi, gəzinti və sonra bira pi...",0,1,1


We should shuffle one more time,because first 9000 rows contains only original tweets.We must shuffle the data to make the learning process for LR more better

In [ ]:
np.random.seed(130)
shuffled_overall=overall.sample(frac=1).reset_index(drop=True)
shuffled_overall

,id,tweet,sarcasm,humor,insult
0,3706.0,do you ever go to school confident in what you...,0,0,0
1,878.0,@annetut25... act as as ice you know. We can j...,0,0,1
2,3625.0,Double Sexy Package Is The Best Type Of Package,0,1,1
3,798.0,And that seals it for me 'cause this woman has...,0,0,0
4,8410.0,I lost the voicemail that Brody left me in Feb...,0,1,1
...,...,...,...,...,...
9995,7691.0,I had all weekend to read the shit my quiz is ...,0,0,0
9996,NaN,Erkən oyandıqdan sonra yenidən yuxuya getmək d...,0,1,0
9997,7966.0,I hate when girls act like they know me person...,0,0,1
9998,7286.0,If a mf don't care if you eat the whole day th...,0,0,1


So,we have 3 ways  how to make model classify the text.
First one is:

*   Take the overall dataset and tokenize the text in word level



In [ ]:
import tensorflow as tf
t=tf.keras.preprocessing.text.Tokenizer(
    num_words=20000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,
    split=' ', char_level=False, oov_token = 'sabir')

In [ ]:
tweets=np.array(overall['tweet'])
labels=np.array(overall[['sarcasm','humor','insult']])

In [ ]:
t.fit_on_texts(tweets)

In [ ]:
sorted(t.word_index.items(),key=lambda x:x[1])

[('sabir', 1),
 ('i', 2),
 ('to', 3),
 ('the', 4),
 ('you', 5),
 ('a', 6),
 ('and', 7),
 ('my', 8),
 ('of', 9),
 ('in', 10),
 ('is', 11),
 ('for', 12),
 ('it', 13),
 ('love', 14),
 ('that', 15),
 ('so', 16),
 ('me', 17),
 ('have', 18),
 ('on', 19),
 ('this', 20),
 ('be', 21),
 ('with', 22),
 ("i'm", 23),
 ('your', 24),
 ('just', 25),
 ('but', 26),
 ('not', 27),
 ('like', 28),
 ('at', 29),
 ("don't", 30),
 ('all', 31),
 ('do', 32),
 ('when', 33),
 ('are', 34),
 ('day', 35),
 ('if', 36),
 ('hate', 37),
 ('get', 38),
 ('was', 39),
 ('good', 40),
 ('up', 41),
 ('can', 42),
 ('how', 43),
 ('😂', 44),
 ('people', 45),
 ('happy', 46),
 ('they', 47),
 ('great', 48),
 ('u', 49),
 ('out', 50),
 ('about', 51),
 ('much', 52),
 ("can't", 53),
 ('an', 54),
 ('we', 55),
 ('what', 56),
 ("it's", 57),
 ('one', 58),
 ('know', 59),
 ('sarcasm', 60),
 ('really', 61),
 ('no', 62),
 ('as', 63),
 ('time', 64),
 ('wait', 65),
 ('shit', 66),
 ('from', 67),
 ('go', 68),
 ('who', 69),
 ('had', 70),
 ('he', 71),
 

In [ ]:
tokenized=t.texts_to_sequences(tweets)

In [ ]:
sum(map(len, tokenized))/len(tokenized)

15.628

In [ ]:
import tensorflow.keras.preprocessing.sequence as seq

ready_tweets = seq.pad_sequences(tokenized, maxlen=8, padding='post', truncating='post')

In [ ]:
ready_tweets.shape

(10000, 8)

In [ ]:
x_train=ready_tweets[:8000]
x_test=ready_tweets[8000:]
train_labels=labels[:8000]
test_labels=labels[8000:]

We can also take these three classes and make them binary.Lets compare binary and original labels to see,in which case model would perform better

sarcasm,sarcasm and humor,humor,humor and insult,insult,sarcasm and insult,none

In [ ]:
train_labels_binary=np.zeros((8000,7))
test_labels_binary=np.zeros((2000,7))
for i in range(train_labels.shape[0]):
  if train_labels[i][0]==1 and train_labels[i][1]==0 and train_labels[i][2]==0:
    train_labels_binary[i,0]=1
    train_labels_binary[i,1]=0
    train_labels_binary[i,2]=0
    train_labels_binary[i,3]=0
    train_labels_binary[i,4]=0
    train_labels_binary[i,5]=0
    train_labels_binary[i,6]=0
  elif train_labels[i][0]==1 and train_labels[i][1]==1 and train_labels[i][2]==0:
    train_labels_binary[i,0]=0
    train_labels_binary[i,1]=1
    train_labels_binary[i,2]=0
    train_labels_binary[i,3]=0
    train_labels_binary[i,4]=0
    train_labels_binary[i,5]=0
    train_labels_binary[i,6]=0
  elif train_labels[i][0]==0 and train_labels[i][1]==1 and train_labels[i][2]==0:
    train_labels_binary[i,0]=0
    train_labels_binary[i,1]=0
    train_labels_binary[i,2]=1
    train_labels_binary[i,3]=0
    train_labels_binary[i,4]=0
    train_labels_binary[i,5]=0
    train_labels_binary[i,6]=0
  elif train_labels[i][0]==0 and train_labels[i][1]==1 and train_labels[i][2]==1:
    train_labels_binary[i,0]=0
    train_labels_binary[i,1]=0
    train_labels_binary[i,2]=0
    train_labels_binary[i,3]=1
    train_labels_binary[i,4]=0
    train_labels_binary[i,5]=0
    train_labels_binary[i,6]=0
  elif train_labels[i][0]==0 and train_labels[i][1]==0 and train_labels[i][2]==1:
    train_labels_binary[i,0]=0
    train_labels_binary[i,1]=0
    train_labels_binary[i,2]=0
    train_labels_binary[i,3]=0
    train_labels_binary[i,4]=1
    train_labels_binary[i,5]=0
    train_labels_binary[i,6]=0
  elif train_labels[i][0]==1 and train_labels[i][1]==0 and train_labels[i][2]==1:
    train_labels_binary[i,0]=0
    train_labels_binary[i,1]=0
    train_labels_binary[i,2]=0
    train_labels_binary[i,3]=0
    train_labels_binary[i,4]=0
    train_labels_binary[i,5]=1
    train_labels_binary[i,6]=0
  elif train_labels[i][0]==0 and train_labels[i][1]==0 and train_labels[i][2]==0:
    train_labels_binary[i,0]=0
    train_labels_binary[i,1]=0
    train_labels_binary[i,2]=0
    train_labels_binary[i,3]=0
    train_labels_binary[i,4]=0
    train_labels_binary[i,5]=0
    train_labels_binary[i,6]=1

In [ ]:
train_labels_binary=train_labels_binary.astype(int)

Because I have multioutput and LR can't deal with multioutput,we need to use OnevsRest approach

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
clf=OneVsRestClassifier(LogisticRegression(random_state=45)).fit(x_train,train_labels_binary)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 1 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to

In [ ]:
for i in range(test_labels.shape[0]):
  if test_labels[i][0]==1 and test_labels[i][1]==0 and test_labels[i][2]==0:
    test_labels_binary[i,0]=1
    test_labels_binary[i,1]=0
    test_labels_binary[i,2]=0
    test_labels_binary[i,3]=0
    test_labels_binary[i,4]=0
    test_labels_binary[i,5]=0
    test_labels_binary[i,6]=0
  elif test_labels[i][0]==1 and test_labels[i][1]==1 and test_labels[i][2]==0:
    test_labels_binary[i,0]=0
    test_labels_binary[i,1]=1
    test_labels_binary[i,2]=0
    test_labels_binary[i,3]=0
    test_labels_binary[i,4]=0
    test_labels_binary[i,5]=0
    test_labels_binary[i,6]=0
  elif test_labels[i][0]==0 and test_labels[i][1]==1 and test_labels[i][2]==0:
    test_labels_binary[i,0]=0
    test_labels_binary[i,1]=0
    test_labels_binary[i,2]=1
    test_labels_binary[i,3]=0
    test_labels_binary[i,4]=0
    test_labels_binary[i,5]=0
    test_labels_binary[i,6]=0
  elif test_labels[i][0]==0 and test_labels[i][1]==1 and test_labels[i][2]==1:
    test_labels_binary[i,0]=0
    test_labels_binary[i,1]=0
    test_labels_binary[i,2]=0
    test_labels_binary[i,3]=1
    test_labels_binary[i,4]=0
    test_labels_binary[i,5]=0
    test_labels_binary[i,6]=0
  elif test_labels[i][0]==0 and test_labels[i][1]==0 and test_labels[i][2]==1:
    test_labels_binary[i,0]=0
    test_labels_binary[i,1]=0
    test_labels_binary[i,2]=0
    test_labels_binary[i,3]=0
    test_labels_binary[i,4]=1
    test_labels_binary[i,5]=0
    test_labels_binary[i,6]=0
  elif test_labels[i][0]==1 and test_labels[i][1]==0 and test_labels[i][2]==1:
    test_labels_binary[i,0]=0
    test_labels_binary[i,1]=0
    test_labels_binary[i,2]=0
    test_labels_binary[i,3]=0
    test_labels_binary[i,4]=0
    test_labels_binary[i,5]=1
    test_labels_binary[i,6]=0
  elif test_labels[i][0]==0 and test_labels[i][1]==0 and test_labels[i][2]==0:
    test_labels_binary[i,0]=0
    test_labels_binary[i,1]=0
    test_labels_binary[i,2]=0
    test_labels_binary[i,3]=0
    test_labels_binary[i,4]=0
    test_labels_binary[i,5]=0
    test_labels_binary[i,6]=1

In [ ]:
clf.score(x_test,test_labels_binary)

0.0515

5% is really bad result,so the theory,that more labels you have,the less is accuracy,is absolutely true

In [ ]:
np.random.seed(349)
clf_2=OneVsRestClassifier(LogisticRegression(random_state=38)).fit(x_train,train_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
clf_2.score(x_test,test_labels)

0.186

When we use three labels,the result is much better.We have 6 different outputs in this case,so this means,that model has learnt something

100/6=16.6666666667


In [ ]:
sentence=['lumos Azərbaycanin ən böyük kursu sayilir']
sentence=t.texts_to_sequences(sentence)
sentence=seq.pad_sequences(sentence, maxlen=8, padding='post', truncating='post')
clf_2.predict(sentence)

array([[0, 0, 0]])

It is not sarcasm,not humor and not insult.Even ML says,that lumos is the best!

In [ ]:
sentence=['Ilhami bütün Azerbaycan söyür']
sentence=t.texts_to_sequences(sentence)
sentence=seq.pad_sequences(sentence, maxlen=8, padding='post', truncating='post')
clf_2.predict(sentence)

array([[0, 1, 1]])

ML says that it is a humor and insult.Cool!

In [ ]:
sentence=['sabir will have a  girlfriend']
sentence=t.texts_to_sequences(sentence)
sentence=seq.pad_sequences(sentence, maxlen=8, padding='post', truncating='post')
clf_2.predict(sentence)

array([[0, 0, 1]])

So,it is  insult!

In [ ]:
sentence=['Sexual assault is what makes Azerbaijan so unique!']
sentence=t.texts_to_sequences(sentence)
sentence=seq.pad_sequences(sentence, maxlen=8, padding='post', truncating='post')
clf_2.predict(sentence)

array([[0, 1, 1]])

Humor and insult!

The second way:

*   Take the overall dataset and tokenize the text in char level.




Disadvantage of this method is these two languages have lots of same letters,so it will be confusing for LR to learn patterns



In [ ]:
import tensorflow as tf
char_tokenizer=tf.keras.preprocessing.text.Tokenizer(
    num_words=60, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,
    split=' ', char_level=True, oov_token = 'sabir')

In [ ]:
tweets=np.array(overall['tweet'])
labels=np.array(overall[['sarcasm','humor','insult']])

In [ ]:
char_tokenizer.fit_on_texts(tweets)

In [ ]:
sorted(char_tokenizer.word_index.items(),key=lambda x:x[1])

[('sabir', 1),
 (' ', 2),
 ('e', 3),
 ('a', 4),
 ('t', 5),
 ('o', 6),
 ('i', 7),
 ('n', 8),
 ('s', 9),
 ('r', 10),
 ('l', 11),
 ('h', 12),
 ('d', 13),
 ('u', 14),
 ('y', 15),
 ('m', 16),
 ('g', 17),
 ('c', 18),
 ('w', 19),
 ('b', 20),
 ('f', 21),
 ('p', 22),
 ('k', 23),
 ('v', 24),
 ('.', 25),
 ("'", 26),
 ('ə', 27),
 ('@', 28),
 ('#', 29),
 (',', 30),
 ('!', 31),
 ('x', 32),
 ('j', 33),
 ('ı', 34),
 ('z', 35),
 ('ü', 36),
 ('q', 37),
 ('?', 38),
 ('"', 39),
 ('1', 40),
 ('ş', 41),
 ('-', 42),
 (':', 43),
 ('_', 44),
 ('2', 45),
 ('0', 46),
 ('ç', 47),
 ('&', 48),
 ('😂', 49),
 ('5', 50),
 ('3', 51),
 ('ğ', 52),
 ('ö', 53),
 (')', 54),
 ('4', 55),
 ('9', 56),
 ('❤', 57),
 ('️', 58),
 ('/', 59),
 ('(', 60),
 ('6', 61),
 ('7', 62),
 ('8', 63),
 ('😍', 64),
 ('😭', 65),
 ('😘', 66),
 ('😊', 67),
 ('*', 68),
 ('😩', 69),
 (';', 70),
 ('̇', 71),
 ('😒', 72),
 ('$', 73),
 ('%', 74),
 ('😁', 75),
 ('☺', 76),
 ('+', 77),
 ('—', 78),
 ('~', 79),
 ('�', 80),
 ('😔', 81),
 ('😷', 82),
 ('💙', 83),
 ('😑', 84

In [ ]:
tokenized=char_tokenizer.texts_to_sequences(tweets)

In [ ]:
num=[]
for sentence in tokenized:
  num.append(len(sentence))

In [ ]:
np.percentile(num,50)

85.0

In [ ]:
import tensorflow.keras.preprocessing.sequence as seq

overall_tweets = seq.pad_sequences(tokenized, maxlen=85, padding='post', truncating='post')

In [ ]:
x_train=overall_tweets[:8000]
x_test=overall_tweets[8000:]
train_labels=labels[:8000]
test_labels=labels[8000:]

In [ ]:
clf_char=OneVsRestClassifier(LogisticRegression(random_state=78)).fit(x_train,train_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
clf_char.score(x_test,test_labels)

0.197

Result is slightly better than previous one,but this method is not the best

The third way is:

*   Take two separate datasets(original and translated tweets) and train them separately



Advantage of this method is model learns these two languages separately,so model avoids any confusion




In [ ]:
azerbaijani_tweets=np.array(azerbaijani_dataset['tweet'])
labels=np.array(azerbaijani_dataset[['sarcasm','humor','insult']])

In [ ]:
import tensorflow as tf
t=tf.keras.preprocessing.text.Tokenizer(
    num_words=50, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,
    split=' ', char_level=True, oov_token = 'sabir')

In [ ]:
t.fit_on_texts(azerbaijani_tweets)

In [ ]:
tokenized=t.texts_to_sequences(azerbaijani_tweets)

In [ ]:
num=[]
for sentence in tokenized:
  num.append(len(sentence))
np.percentile(num,50)

88.0

In [ ]:
import tensorflow.keras.preprocessing.sequence as seq

ready_tweets = seq.pad_sequences(tokenized, maxlen=88, padding='post', truncating='post')

In [ ]:
x_train=ready_tweets[:800]
x_test=ready_tweets[800:]
train_labels=labels[:800]
test_labels=labels[800:]

In [ ]:
clf_char_az=OneVsRestClassifier(LogisticRegression(random_state=78)).fit(x_train,train_labels)

In [ ]:
clf_char_az.score(x_test,test_labels)

0.135

In [ ]:
original_tweets=np.array(all_comments['tweet'])
labels=np.array(all_comments[['sarcasm','humor','insult']])

In [ ]:
t=tf.keras.preprocessing.text.Tokenizer(
    num_words=50, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,
    split=' ', char_level=True, oov_token = 'sabir')

In [ ]:
t.fit_on_texts(original_tweets)

In [ ]:
tokenized=t.texts_to_sequences(original_tweets)

In [ ]:
num=[]
for sentence in tokenized:
  num.append(len(sentence))
np.percentile(num,50)

85.0

In [ ]:
ready_tweets = seq.pad_sequences(tokenized, maxlen=85, padding='post', truncating='post')

In [ ]:
x_train=ready_tweets[:7500]
x_test=ready_tweets[7500:]
train_labels=labels[:7500]
test_labels=labels[7500:]

(1500, 3)

In [ ]:
clf_char_en=OneVsRestClassifier(LogisticRegression(random_state=78)).fit(x_train,train_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
clf_char_en.score(x_test,test_labels)

0.25466666666666665

This method is more better,and we have 25% accuracy,which is really great

In all of these cases I used imbalanced dataset.Lets balance it!I will use 5000 Azerbaijani samples and 5000 original samples

In [ ]:
all_comments=pd.read_csv('/content/sarcasmania-dataset.csv')

In [ ]:
np.random.seed(128)
shuffled_comments=all_comments.sample(frac=1).reset_index(drop=True)

In [ ]:
shuffled_comments=shuffled_comments.iloc[:5000,:]

In [ ]:
az_tweets=[]
for tweet in shuffled_comments['tweet']:
  translation=translator.translate(tweet,src='en',dest='az')
  az_tweets.append(translation.text)

In [ ]:
sarcasm=shuffled_comments.iloc[:5000,2]
humor=shuffled_comments.iloc[:5000,3]
insult=shuffled_comments.iloc[:5000,4]

In [ ]:
azerbaijani_dataset=pd.DataFrame({'tweet':az_tweets,'sarcasm':sarcasm,'humor':humor,'insult':insult})

In [ ]:
all_comments=all_comments.iloc[:5000,:]

In [ ]:
overall=pd.concat([all_comments,azerbaijani_dataset])

In [ ]:
np.random.seed(130)
shuffled_overall=overall.sample(frac=1).reset_index(drop=True)

In [ ]:
t=tf.keras.preprocessing.text.Tokenizer(
    num_words=60, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,
    split=' ', char_level=True, oov_token = 'sabir')

In [ ]:
tweets=np.array(overall['tweet'])
labels=np.array(overall[['sarcasm','humor','insult']])

In [ ]:
t.fit_on_texts(tweets)

In [ ]:
tokenized=t.texts_to_sequences(tweets)

In [ ]:
sum(map(len, tokenized))/len(tokenized)

89.4417

In [ ]:
ready_tweets = seq.pad_sequences(tokenized, maxlen=89, padding='post', truncating='post')

In [ ]:
x_train=ready_tweets[:8000]
x_test=ready_tweets[8000:]
train_labels=labels[:8000]
test_labels=labels[8000:]

In [ ]:
clf=OneVsRestClassifier(LogisticRegression(random_state=45)).fit(x_train,train_labels)

In [ ]:
clf.score(x_test,test_labels)

0.1395

The reason why results are not so good is because dataset was not so much good.Or maybe we need much more samples.